In [9]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization

In [2]:
file_path = '../Data/data_pre_procesada.csv'
data = pd.read_csv(file_path)
data.head()

,Coleccion,Referencia,Descripcion,Color,No.Pedido,Tot_Piezas,Costo_Fabrica,Precio_Venta,Pais_Producto,Cta_Cliente,Nombre_Cliente,Estatus,Plataforma,familia,Pais_Producto_fil,Cta_Cliente_final,Temporada,Nueva_Coleccion
0,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5597,6.0,15.65,25.0,china,2313076,"RIPOSTO,S.A.",EN SISTEMA,ASIA,RK,china,2313076,FW,2016-11-01
1,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5630,2.0,15.65,25.0,china,2224074,ICOSAL S.A. DE C.V.,EN SISTEMA,ASIA,RK,china,2224074,FW,2016-11-01
2,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5631,2.0,15.65,25.0,china,2224090,"ICOSAL,S.A. AEROPUERT",EN SISTEMA,ASIA,RK,china,2224090,FW,2016-11-01
3,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5635,5.0,15.65,25.0,china,2209051,"INVERSIONES MONTIJO, S.A.",EN SISTEMA,ASIA,RK,china,2209051,FW,2016-11-01
4,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,031,5597,6.0,15.65,25.0,china,2313076,"RIPOSTO,S.A.",EN SISTEMA,ASIA,RK,china,2313076,FW,2016-11-01


In [5]:
# Agrupando por temporada, cliente y familia
seasonal_data = data.groupby(['Nueva_Coleccion', 'Cta_Cliente_final', 'familia']).agg({'Tot_Piezas':'sum', 'Precio_Venta':'mean'}).reset_index()

In [13]:
def series_to_supervised(df, n_steps):
    X, y = [], []
    # Iterar por cada combinación única de 'Cta_Cliente_final' y 'familia'
    for cliente, cliente_df in df.groupby(['Cta_Cliente_final', 'familia']):
        # Asegurarse de que los datos estén ordenados por 'Nueva_Coleccion'
        cliente_df = cliente_df.sort_values('Nueva_Coleccion')
        # Extraer solo las columnas de interés
        values = cliente_df[['Tot_Piezas', 'Precio_Venta']].values
        for i in range(len(values) - n_steps):
            # Crear secuencias de X e y
            X.append(values[i:(i + n_steps), :])
            y.append(values[i + n_steps, 0])  # Asumiendo que Tot_Piezas es la primera columna
    return np.array(X), np.array(y)

# Definir n_steps
n_steps = 8  # Ejemplo, ajusta según necesidad
# Preparar los datos
X, y = series_to_supervised(seasonal_data, n_steps)

print(f"X shape: {X.shape}")  # Debería ser (n_samples, n_steps, n_features)
print(f"y shape: {y.shape}")  # Debería ser (n_samples,)

X shape: (1002, 8, 2)
y shape: (1002,)


In [14]:
seasonal_data.head()

,Nueva_Coleccion,Cta_Cliente_final,familia,Tot_Piezas,Precio_Venta
0,2016-05-01,2000006,AH,30.0,75.0
1,2016-05-01,2000006,CF,22.0,62.0
2,2016-05-01,2000006,CH,24.0,67.0
3,2016-05-01,2000006,CJ,8.0,31.0
4,2016-05-01,2000006,EF,8.0,73.0


In [15]:
# Aplanar los datos de entrada para escalar
n_input = X.shape[1] * X.shape[2]  # n_steps * n_features
X_flattened = X.reshape((X.shape[0], n_input))

# Escalar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_flattened)

# Volver a dar forma a los datos escalados a la forma original
X_scaled = X_scaled.reshape((X.shape[0], X.shape[1], X.shape[2]))

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [16]:
model = Sequential([
    LSTM(50, activation='relu', input_shape=(n_steps, 2)),  # 2 por Tot_Piezas y Precio_Venta
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')

In [17]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/100
20/20 [==============================] - 1s 18ms/step - loss: 238255.2812 - val_loss: 386232.7500
Epoch 2/100
20/20 [==============================] - 0s 7ms/step - loss: 237817.7969 - val_loss: 385416.8750
Epoch 3/100
20/20 [==============================] - 0s 7ms/step - loss: 233280.5312 - val_loss: 364807.6875
Epoch 4/100
20/20 [==============================] - 0s 8ms/step - loss: 189762.7031 - val_loss: 263900.6562
Epoch 5/100
20/20 [==============================] - 0s 7ms/step - loss: 178992.5156 - val_loss: 207556.3906
Epoch 6/100
20/20 [==============================] - 0s 9ms/step - loss: 148457.7031 - val_loss: 254985.4219
Epoch 7/100
20/20 [==============================] - 0s 15ms/step - loss: 133759.3594 - val_loss: 190227.2188
Epoch 8/100
20/20 [==============================] - 0s 16ms/step - loss: 120946.9219 - val_loss: 182507.0000
Epoch 9/100
20/20 [==============================] - 0s 16ms/step - loss: 112977.1406 - val_loss: 164181.8438
Epoch 10/100
20

Epoch 76/100
20/20 [==============================] - 0s 8ms/step - loss: 69692.4766 - val_loss: 141691.9062
Epoch 77/100
20/20 [==============================] - 0s 9ms/step - loss: 71644.2578 - val_loss: 137866.2344
Epoch 78/100
20/20 [==============================] - 0s 8ms/step - loss: 67617.2969 - val_loss: 145991.7500
Epoch 79/100
20/20 [==============================] - 0s 9ms/step - loss: 66176.3281 - val_loss: 140325.2344
Epoch 80/100
20/20 [==============================] - 0s 9ms/step - loss: 67012.0078 - val_loss: 140970.0312
Epoch 81/100
20/20 [==============================] - 0s 10ms/step - loss: 65536.5859 - val_loss: 138100.5000
Epoch 82/100
20/20 [==============================] - 0s 14ms/step - loss: 68709.9609 - val_loss: 128384.7109
Epoch 83/100
20/20 [==============================] - 0s 12ms/step - loss: 68962.1016 - val_loss: 162572.0156
Epoch 84/100
20/20 [==============================] - 0s 12ms/step - loss: 65926.0469 - val_loss: 140000.2031
Epoch 85/100
20

In [18]:
# Hacer predicciones
y_pred = model.predict(X_test)

# Calcular métricas de evaluación
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R^2: {r2}")

7/7 [==============================] - 0s 4ms/step
MAE: 145.12513728462048
MSE: 179584.35520735275
RMSE: 423.7739435210154
R^2: 0.5349373170189781
